# 🖥️ Manipulating Field data

Because Parcels `Field` objects are built on top of `xarray` DataArrays, you can leverage the powerful data manipulation capabilities of `xarray` to preprocess or modify your field data before using it in particle simulations. This tutorial provides some examples of how to leverage that power.

## Summing Fields
In some applications, you may want to sum multiple fields together to create a combined effect on particle movement. For example, you might want to combine ocean currents with wind-driven surface drift. This tutorial demonstrates how to sum multiple fields in Parcels and visualize the resulting particle trajectories.

We base this tutorial on the example provided in the [Parcels Kernel loop explanation](explanation_kernelloop.md). There, we combined two kernels to simulate the joint effect of currents and winds.

However, we can also do that in one kernel, by combining the fields directly, using `xarray` operations.

We start with loading the necessary libraries and data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

# Load the CopernicusMarine data in the Agulhas region from the example_datasets
example_dataset_folder = parcels.download_example_dataset(
    "CopernicusMarine_data_for_Argo_tutorial"
)

ds_fields = xr.open_mfdataset(f"{example_dataset_folder}/*.nc", combine="by_coords")
ds_fields.load()  # load the dataset into memory

# Create an idealised wind field and add it to the dataset
tdim, ydim, xdim = (
    len(ds_fields.time),
    len(ds_fields.latitude),
    len(ds_fields.longitude),
)
ds_fields["UWind"] = xr.DataArray(
    data=np.ones((tdim, ydim, xdim)) * np.sin(ds_fields.latitude.values)[None, :, None],
    coords=[ds_fields.time, ds_fields.latitude, ds_fields.longitude],
)

ds_fields["VWind"] = xr.DataArray(
    data=np.zeros((tdim, ydim, xdim)),
    coords=[ds_fields.time, ds_fields.latitude, ds_fields.longitude],
)

Now here comes the trick: we can simply sum the fields together using `xarray` operations

In [ ]:
# Combine ocean currents and wind fields
ds_fields["U"] = ds_fields["uo"] + ds_fields["UWind"]
ds_fields["V"] = ds_fields["vo"] + ds_fields["VWind"]

```{note}
Combining fields in this way assumes that the fields are defined on the same grid and have compatible dimensions. Ensure that the fields you are summing are aligned correctly to avoid unexpected results.
```

We can then run the same simulation as in the [Kernel loop explanation tutorial](explanation_kernelloop.md), but now using the combined fields. We only need the default advection kernel now, since the effects of both currents and winds are already included in the summed fields.

In [ ]:
fields = {
    "U": ds_fields["U"],
    "V": ds_fields["V"],
}
ds_fset = parcels.convert.copernicusmarine_to_sgrid(fields=fields)
fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)

npart = 10
z = np.repeat(ds_fields.depth[0].values, npart)
lons = np.repeat(31, npart)
lats = np.linspace(-32.5, -30.5, npart)

pset = parcels.ParticleSet(fieldset, pclass=parcels.Particle, z=z, lat=lats, lon=lons)
output_file = parcels.ParticleFile(
    store="summed_advection_wind.zarr", outputdt=np.timedelta64(6, "h")
)
pset.execute(
    [parcels.kernels.AdvectionRK4],
    runtime=np.timedelta64(5, "D"),
    dt=np.timedelta64(1, "h"),
    output_file=output_file,
)

We can then plot the trajectories, and confirm that they particles move the same as in the [Kernel loop explanation tutorial](explanation_kernelloop.md), where we combined the effects of currents and winds using two separate kernels.

In [ ]:
# Plot the resulting particle trajectories overlapped for both cases
summed_advection_wind = xr.open_zarr("summed_advection_wind.zarr")
plt.plot(summed_advection_wind.lon.T, summed_advection_wind.lat.T, "-")
plt.show()